# Reasoning Frameworks Without Libraries

This notebook demonstrates how to implement advanced reasoning frameworks for LLM agents without relying on specialized libraries. We'll explore:

- ReAct pattern implementation from scratch
- Building a planning-execution loop
- Reflection and self-correction mechanisms
- Implementing chain-of-thought in raw prompts

By building these reasoning patterns from the ground up, you'll gain a deeper understanding of how they work and how to customize them for your specific use cases.

## 1. Setup and Configuration

First, let's set up our environment and import necessary libraries.

In [1]:
# Install required packages
!pip install python-dotenv requests tenacity

In [16]:
import os
import sys
import json
import time
import re
import uuid
from typing import Dict, List, Any, Optional, Union, Tuple, Callable
from dotenv import load_dotenv

# Add parent directory to path to import utility functions
sys.path.append('../../Week1/Day2')

# Import our API utilities
from api_utils import (
    call_openrouter,
    extract_text_response,
    extract_function_call,
    get_available_models
)

# Load environment variables
load_dotenv()

# Verify API key is loaded
api_key = os.getenv("OPENROUTER_API_KEY")
if api_key:
    print("✅ API key loaded successfully!")
    # Show first and last three characters for verification
    masked_key = f"{api_key[:3]}...{api_key[-3:]}" if len(api_key) > 6 else "[key too short]"
    print(f"API key: {masked_key}")
else:
    print("❌ API key not found! Make sure you've created a .env file with your OPENROUTER_API_KEY.")

✅ API key loaded successfully!
API key: sk-...843


## 2. Chain of Thought (CoT) Prompting

Chain of Thought (CoT) is a prompting technique that encourages the model to show its reasoning process step by step. This typically improves performance on complex reasoning tasks by breaking them down into manageable steps.

Let's implement a simple CoT approach without any libraries:

In [14]:
def chain_of_thought(question: str, model: str = "openai/gpt-4o-mini-2024-07-18") -> str:
    """Apply chain-of-thought prompting to encourage step-by-step reasoning.
    
    Args:
        question: The question to answer
        model: The LLM model to use
        
    Returns:
        The model's response with reasoning steps
    """
    # System prompt that encourages step-by-step reasoning
    system_prompt = """
    You are an expert problem-solver who thinks through problems step by step.
    When given a question, break down your thinking process into clear steps:
    1. Understand what the question is asking
    2. Identify the key information and constraints
    3. Consider different approaches to solve the problem
    4. Walk through the solution step by step
    5. Verify the answer to ensure it makes sense
    
    Structure your answers with clear reasoning steps, showing your full thought process.
    Start with "Let me think through this step by step:"
    """
    
    # Make the API call
    response = call_openrouter(
        prompt=question,
        model=model,
        system_prompt=system_prompt,
        temperature=0.3, # Lower temperature for more logical reasoning
        max_tokens=800
    )
    
    if response.get("success", False):
        return extract_text_response(response)
    else:
        return f"Error: {response.get('error', 'Unknown error')}"

# Test with a complex reasoning question
cot_question = "A merchant has 50 fruits in total, consisting of apples, oranges, and bananas. He has twice as many apples as oranges, and 5 more bananas than oranges. How many of each fruit does he have?"

print(f"Question: {cot_question}\n")
print("Chain of Thought Response:")
print(chain_of_thought(cot_question))

Question: A merchant has 50 fruits in total, consisting of apples, oranges, and bananas. He has twice as many apples as oranges, and 5 more bananas than oranges. How many of each fruit does he have?

Chain of Thought Response:
Let me think through this step by step:

1. **Understand what the question is asking**: We need to find out how many apples, oranges, and bananas the merchant has, given the total number of fruits and the relationships between the quantities of each type of fruit.

2. **Identify the key information and constraints**:
   - Total fruits = 50
   - Let the number of oranges be \( x \).
   - The number of apples is twice the number of oranges: \( 2x \).
   - The number of bananas is 5 more than the number of oranges: \( x + 5 \).

3. **Consider different approaches to solve the problem**: We can set up an equation based on the total number of fruits and the relationships given.

4. **Walk through the solution step by step**:
   - We know the total number of fruits:
  

### 2.1 Chain of Thought with Few-Shot Examples

We can further enhance CoT by providing few-shot examples in the prompt. This helps the model understand the expected reasoning pattern.

In [4]:
def few_shot_cot(question: str, model: str = "openai/gpt-4o-mini-2024-07-18") -> str:
    """Apply chain-of-thought prompting with few-shot examples.
    
    Args:
        question: The question to answer
        model: The LLM model to use
        
    Returns:
        The model's response with reasoning steps
    """
    # Create a prompt with few-shot examples
    few_shot_prompt = """
    I'll solve each problem step by step.
    
    Problem: If John has 5 apples and eats 2, then buys 3 more, how many apples does he have?
    Solution: Let me think through this step by step.
    1. Initially, John has 5 apples.
    2. He eats 2 apples, so he has 5 - 2 = 3 apples left.
    3. Then he buys 3 more apples, so he has 3 + 3 = 6 apples total.
    4. Therefore, John has 6 apples.
    
    Problem: A rectangle has a perimeter of 24 cm and a width of 4 cm. What is its area?
    Solution: Let me think through this step by step.
    1. The perimeter of a rectangle is 2 × (length + width).
    2. Given that the perimeter is 24 cm and width is 4 cm.
    3. 24 = 2 × (length + 4)
    4. 12 = length + 4
    5. length = 8 cm
    6. The area of a rectangle is length × width = 8 cm × 4 cm = 32 cm²
    7. Therefore, the area of the rectangle is 32 square centimeters.
    
    Problem: {question}
    Solution: Let me think through this step by step.
    """.format(question=question)
    
    # Make the API call
    response = call_openrouter(
        prompt=few_shot_prompt,
        model=model,
        temperature=0.3,
        max_tokens=800
    )
    
    if response.get("success", False):
        return extract_text_response(response)
    else:
        return f"Error: {response.get('error', 'Unknown error')}"

# Test with the same question to compare results
print(f"Question: {cot_question}\n")
print("Few-Shot Chain of Thought Response:")
print(few_shot_cot(cot_question))

Question: A merchant has 50 fruits in total, consisting of apples, oranges, and bananas. He has twice as many apples as oranges, and 5 more bananas than oranges. How many of each fruit does he have?

Few-Shot Chain of Thought Response:
1. Let the number of oranges be \( x \).
2. According to the problem, the number of apples is \( 2x \) (twice as many apples as oranges).
3. The number of bananas is \( x + 5 \) (5 more bananas than oranges).
4. The total number of fruits is given as 50. Therefore, we can set up the equation:
   \[
   x + 2x + (x + 5) = 50
   \]
5. Simplifying the equation:
   \[
   4x + 5 = 50
   \]
6. Subtracting 5 from both sides:
   \[
   4x = 45
   \]
7. Dividing both sides by 4:
   \[
   x = 11.25
   \]
   (This indicates that there is an issue since the number of fruits should be a whole number.)

Let's re-evaluate the problem. 

1. Let the number of oranges be \( x \).
2. The number of apples is \( 2x \).
3. The number of bananas is \( x + 5 \).
4. The total numb

### 2.2 Let's Compare Different Reasoning Approaches

Let's compare regular prompting, chain of thought, and few-shot CoT on a complex reasoning task.

In [5]:
def regular_prompt(question: str, model: str = "openai/gpt-4o-mini-2024-07-18") -> str:
    """A baseline approach without explicit reasoning instructions."""
    response = call_openrouter(
        prompt=question,
        model=model,
        temperature=0.3,
        max_tokens=500
    )
    
    if response.get("success", False):
        return extract_text_response(response)
    else:
        return f"Error: {response.get('error', 'Unknown error')}"

# A more complex reasoning question
complex_question = """
A group of friends - Alice, Bob, Charlie, and Diana - are solving a puzzle that requires arranging 4 colored balls (red, blue, green, yellow) in a specific order.
Given these clues:
1. The red ball comes before the blue ball, but not immediately before.
2. The yellow ball is either the first or the last.
3. Charlie's ball is immediately after Diana's ball.
4. Alice has the green ball, and Bob doesn't have a ball at either end.

What is the correct order of the balls from left to right, and who has which ball?
"""

print("\nComparing different reasoning approaches on a complex logic puzzle:")
print("=" * 80)
print(f"Question: {complex_question}\n")

print("\n1. Regular Prompting:")
print("-" * 40)
regular_response = regular_prompt(complex_question)
print(regular_response)

print("\n2. Chain of Thought:")
print("-" * 40)
cot_response = chain_of_thought(complex_question)
print(cot_response)

print("\n3. Few-Shot Chain of Thought:")
print("-" * 40)
few_shot_response = few_shot_cot(complex_question)
print(few_shot_response)


Comparing different reasoning approaches on a complex logic puzzle:
Question: 
A group of friends - Alice, Bob, Charlie, and Diana - are solving a puzzle that requires arranging 4 colored balls (red, blue, green, yellow) in a specific order.
Given these clues:
1. The red ball comes before the blue ball, but not immediately before.
2. The yellow ball is either the first or the last.
3. Charlie's ball is immediately after Diana's ball.
4. Alice has the green ball, and Bob doesn't have a ball at either end.

What is the correct order of the balls from left to right, and who has which ball?



1. Regular Prompting:
----------------------------------------
To solve the puzzle, let's analyze the clues step by step.

1. **Clue 1**: The red ball comes before the blue ball, but not immediately before. This means there must be at least one ball between the red and blue balls.

2. **Clue 2**: The yellow ball is either the first or the last. This gives us two possible positions for the yellow bal

## 3. ReAct Pattern Implementation

The ReAct (Reasoning+Acting) pattern combines reasoning with action to tackle complex tasks. Let's implement this pattern from scratch without relying on libraries.

### 3.1 First, Let's Define Some Tools

We'll create a few simple tools that our ReAct agent can use.

In [6]:
# Simple Wikipedia-like search tool
def search_wiki(query: str) -> str:
    """Simulates a Wikipedia search."""
    # In a real application, this would call a search API
    wiki_db = {
        "python": "Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability. Python is dynamically typed and garbage-collected. It was created by Guido van Rossum in 1991. Key features include significant whitespace, dynamic typing, and automatic memory management.",
        "neural network": "A neural network is a series of algorithms that endeavors to recognize relationships in a dataset through a process that mimics how the human brain operates. Neural networks are a set of algorithms modeled loosely after the human brain, designed to recognize patterns. They interpret sensory data through machine perception, labeling, or clustering raw input.",
        "machine learning": "Machine learning is a branch of artificial intelligence and computer science that focuses on using data and algorithms to imitate the way that humans learn, gradually improving its accuracy. The primary aim is to develop computer systems that can learn from data without being explicitly programmed.",
        "quantum computing": "Quantum computing is a type of computation that harnesses the collective properties of quantum states, such as superposition, interference, and entanglement, to perform calculations. The devices that perform quantum computations are known as quantum computers.",
        "climate change": "Climate change refers to significant, long-term changes in the global climate. The primary cause is human activities, particularly the burning of fossil fuels, which adds heat-trapping gases to Earth's atmosphere. The consequences include rising temperatures, extreme weather events, shifting wildlife populations and habitats, rising seas, and more."
    }
    
    # Simple keyword matching
    results = []
    query_lower = query.lower()
    
    for key, value in wiki_db.items():
        if key in query_lower or query_lower in key:
            results.append(f"Entry: {key.title()}\n{value}")
    
    if results:
        return "\n\n".join(results)
    else:
        return f"No results found for '{query}'. Try a different search term."

# Calculator tool
def calculator(expression: str) -> str:
    """Evaluates a mathematical expression."""
    try:
        # Warning: eval can be dangerous in production environments
        # For a real application, use a safer approach
        result = eval(expression)
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {str(e)}. Please check the expression format."

# Data lookup tool
def lookup_data(dataset: str, query: str) -> str:
    """Looks up information in a specified dataset."""
    # Simulated datasets
    datasets = {
        "countries": {
            "usa": {"capital": "Washington D.C.", "population": "331 million", "currency": "US Dollar"},
            "japan": {"capital": "Tokyo", "population": "126 million", "currency": "Japanese Yen"},
            "germany": {"capital": "Berlin", "population": "83 million", "currency": "Euro"},
            "nigeria": {"capital": "Abuja", "population": "211 million", "currency": "Nigerian Naira"},
            "brazil": {"capital": "Brasilia", "population": "214 million", "currency": "Brazilian Real"}
        },
        "planets": {
            "mercury": {"position": 1, "type": "Terrestrial", "moons": 0, "rings": "No"},
            "venus": {"position": 2, "type": "Terrestrial", "moons": 0, "rings": "No"},
            "earth": {"position": 3, "type": "Terrestrial", "moons": 1, "rings": "No"},
            "mars": {"position": 4, "type": "Terrestrial", "moons": 2, "rings": "No"},
            "jupiter": {"position": 5, "type": "Gas Giant", "moons": 79, "rings": "Yes"},
            "saturn": {"position": 6, "type": "Gas Giant", "moons": 82, "rings": "Yes"},
            "uranus": {"position": 7, "type": "Ice Giant", "moons": 27, "rings": "Yes"},
            "neptune": {"position": 8, "type": "Ice Giant", "moons": 14, "rings": "Yes"}
        },
        "elements": {
            "hydrogen": {"symbol": "H", "atomic number": 1, "category": "Nonmetal"},
            "helium": {"symbol": "He", "atomic number": 2, "category": "Noble gas"},
            "carbon": {"symbol": "C", "atomic number": 6, "category": "Nonmetal"},
            "oxygen": {"symbol": "O", "atomic number": 8, "category": "Nonmetal"},
            "gold": {"symbol": "Au", "atomic number": 79, "category": "Transition metal"},
            "uranium": {"symbol": "U", "atomic number": 92, "category": "Actinide"}
        }
    }
    
    # Check if dataset exists
    if dataset.lower() not in datasets:
        return f"Dataset '{dataset}' not found. Available datasets: {', '.join(datasets.keys())}"
    
    data = datasets[dataset.lower()]
    query_lower = query.lower()
    
    # Check if specific item is requested
    for key, value in data.items():
        if query_lower == key or query_lower in key:
            result = [f"{key.title()}:"] 
            for attr, val in value.items():
                result.append(f"- {attr.title()}: {val}")
            return "\n".join(result)
    
    # If no specific item matched, return a list of available items
    return f"No specific item '{query}' found in '{dataset}'. Available items: {', '.join(data.keys())}"

### 3.2 Raw ReAct Implementation

Now, let's implement the ReAct pattern from scratch, combining reasoning and acting in a cycle.

In [7]:
def raw_react(question: str, tools: Dict[str, Callable], max_iterations: int = 5, model: str = "openai/gpt-4o-mini-2024-07-18") -> Tuple[str, List[Dict[str, Any]]]:
    """Implements the ReAct pattern from scratch without specialized libraries.
    
    Args:
        question: The user's question
        tools: Dictionary mapping tool names to functions
        max_iterations: Maximum number of reasoning-action cycles
        model: LLM model to use
        
    Returns:
        Tuple of (final answer, list of reasoning-action steps)
    """
    # Initial prompt explaining the format and available tools
    initial_prompt = f"""
    You are an AI assistant that can use tools to answer questions. Follow these steps for each iteration:
    
    1. Think: Consider what information you need and what tools could help.
    2. Action: Choose a tool to use in the format: ACTION: tool_name(parameter1="value", parameter2="value")
    3. Observation: Review the result of the tool use.
    4. Continue this process until you can answer the question.
    
    When you have the answer, respond with: FINAL ANSWER: your complete answer here
    
    Available tools:
    - search_wiki(query): Search for information in a Wikipedia-like database
    - calculator(expression): Evaluate a mathematical expression
    - lookup_data(dataset, query): Look up information in a dataset (available datasets: countries, planets, elements)
    
    Question: {question}
    
    Begin your reasoning process now.
    """
    
    current_prompt = initial_prompt
    iterations = []
    final_answer = ""
    
    for i in range(max_iterations):
        # Get the model's reasoning and action
        response = call_openrouter(
            prompt=current_prompt,
            model=model,
            temperature=0.7,
            max_tokens=800
        )
        
        if not response.get("success", False):
            return f"Error: {response.get('error', 'Unknown error')}", iterations
        
        reasoning = extract_text_response(response)
        
        # Check if we have a final answer
        if "FINAL ANSWER:" in reasoning:
            # Extract the final answer
            final_answer = reasoning.split("FINAL ANSWER:")[1].strip()
            iterations.append({"reasoning": reasoning, "type": "final"})
            break
        
        # Extract the action from the reasoning
        action_match = re.search(r'ACTION:\s*(\w+)\((.+?)\)', reasoning)
        if not action_match:
            # No action found, prompt for a proper action
            current_prompt += f"\n\n{reasoning}\n\nPlease specify an action in the format: ACTION: tool_name(parameter1=\"value\", parameter2=\"value\"), or provide a final answer."
            iterations.append({"reasoning": reasoning, "type": "no_action"})
            continue
        
        tool_name = action_match.group(1).strip()
        args_str = action_match.group(2).strip()
        
        # Parse the arguments
        args = {}
        # Handle named arguments in the format parameter="value"
        for arg_match in re.finditer(r'(\w+)\s*=\s*"([^"]*)"|\s*(\w+)\s*=\s*\'([^\']*)\'', args_str):
            if arg_match.group(1):
                arg_name = arg_match.group(1)
                arg_value = arg_match.group(2)
            else:
                arg_name = arg_match.group(3)
                arg_value = arg_match.group(4)
            args[arg_name] = arg_value
        
        # Execute the tool if it exists
        if tool_name in tools:
            try:
                tool_result = tools[tool_name](**args)
                observation = f"OBSERVATION: {tool_result}"
            except Exception as e:
                observation = f"OBSERVATION: Error executing {tool_name}: {str(e)}"
        else:
            observation = f"OBSERVATION: Tool '{tool_name}' not found. Available tools: {', '.join(tools.keys())}"
        
        # Add the reasoning, action, and observation to the prompt
        current_prompt += f"\n\n{reasoning}\n\n{observation}\n\nContinue your reasoning based on this observation:"
        
        # Record this iteration
        iterations.append({
            "reasoning": reasoning,
            "action": {"tool": tool_name, "args": args},
            "observation": observation,
            "type": "action"
        })
    
    # If we've reached max iterations without a final answer, ask for one
    if not final_answer:
        current_prompt += "\n\nYou've reached the maximum number of iterations. Please provide your final answer now."
        
        response = call_openrouter(
            prompt=current_prompt,
            model=model,
            temperature=0.7,
            max_tokens=500
        )
        
        if response.get("success", False):
            final_reasoning = extract_text_response(response)
            
            # Try to extract a final answer if formatted correctly
            if "FINAL ANSWER:" in final_reasoning:
                final_answer = final_reasoning.split("FINAL ANSWER:")[1].strip()
            else:
                # Otherwise use the whole response
                final_answer = final_reasoning
                
            iterations.append({"reasoning": final_reasoning, "type": "final"})
        else:
            final_answer = f"Error getting final answer: {response.get('error', 'Unknown error')}"
    
    return final_answer, iterations

# Define our tools dictionary
tools_dict = {
    "search_wiki": search_wiki,
    "calculator": calculator,
    "lookup_data": lookup_data
}

# Test the raw ReAct implementation
question1 = "What is the population of the capital of Japan?"
answer1, steps1 = raw_react(question1, tools_dict)

print(f"Question: {question1}\n")
print("ReAct Reasoning Steps:")
for i, step in enumerate(steps1):
    print(f"\nStep {i+1}:")
    print(step["reasoning"])
    if step["type"] == "action":
        print("\n" + step["observation"])

print("\nFinal Answer:")
print(answer1)

Question: What is the population of the capital of Japan?

ReAct Reasoning Steps:

Step 1:
1. Think: I need to find the current population of the capital of Japan, which is Tokyo. A good starting point would be to look up Tokyo's population in a reliable source like a Wikipedia-like database.

2. Action: ACTION: search_wiki("Tokyo population")

3. Observation: I will now check the search results for Tokyo's population.

4. Continue this process until I find the answer. 

Let's perform the search now. 

ACTION: search_wiki("Tokyo population")

OBSERVATION: Error executing search_wiki: search_wiki() missing 1 required positional argument: 'query'

Step 2:
1. Think: It seems there was an error in executing the search. I need to ensure that I use the correct format for the tool. I will try the search again, making sure to properly specify the query.

2. Action: ACTION: search_wiki(query="Tokyo population")

3. Observation: I will now check the search results for Tokyo's population.

Let's 

Let's try another example with a more complex question that will require multiple tool uses:

In [8]:
question2 = "If I have 3 hydrogen atoms and 5 carbon atoms, how many electrons do I have in total?"
answer2, steps2 = raw_react(question2, tools_dict)

print(f"Question: {question2}\n")
print("ReAct Reasoning Steps:")
for i, step in enumerate(steps2):
    print(f"\nStep {i+1}:")
    print(step["reasoning"])
    if step["type"] == "action":
        print("\n" + step["observation"])

print("\nFinal Answer:")
print(answer2)

Question: If I have 3 hydrogen atoms and 5 carbon atoms, how many electrons do I have in total?

ReAct Reasoning Steps:

Step 1:
1. Think: To determine the total number of electrons, I need to know how many electrons are associated with each type of atom. Hydrogen has 1 electron per atom, and carbon has 6 electrons per atom. 

2. Action: I will calculate the total number of electrons by using the formula: (number of hydrogen atoms * electrons per hydrogen) + (number of carbon atoms * electrons per carbon).

   The expression is: (3 * 1) + (5 * 6).

   ACTION: calculator((3 * 1) + (5 * 6))

3. Observation: Now I will evaluate the result of the calculation.

4. Continuing the process: 

After performing the calculation, we get:

- Electrons from hydrogen: 3 * 1 = 3 electrons 
- Electrons from carbon: 5 * 6 = 30 electrons 

Adding these together gives 3 + 30 = 33 electrons.

FINAL ANSWER: You have a total of 33 electrons.

Final Answer:
You have a total of 33 electrons.


## 4. Planning-Execution Loop

Now let's implement a planning-execution loop from scratch. This pattern first creates a plan and then executes each step.

In [20]:
def planning_execution_loop(question: str, tools: Dict[str, Callable], model: str = "openai/gpt-4o-mini-2024-07-18") -> Tuple[str, Dict[str, Any]]:
    """Implements a planning-execution loop from scratch.
    
    Args:
        question: The user's question
        tools: Dictionary mapping tool names to functions
        model: LLM model to use
        
    Returns:
        Tuple of (final answer, dict with plan and execution details)
    """
    # Step 1: Generate a plan
    planning_prompt = f"""
    You are an AI assistant that creates detailed plans to answer questions.
    
    Available tools:
    - search_wiki(query): Search for information in a Wikipedia-like database
    - calculator(expression): Evaluate a mathematical expression
    - lookup_data(dataset, query): Look up information in a dataset (available datasets: countries, planets, elements)
    
    Question: {question}
    
    Create a step-by-step plan to answer this question. For each step, specify:
    1. The tool to use (if any)
    2. The exact parameters to pass to the tool
    3. The purpose of this step
    
    Format each step as:
    
    Step 1: [Purpose of this step]
    Tool: [tool_name or None]
    Parameters: [parameters for the tool, if applicable]
    
    Step 2: ...
    
    Make sure your plan is comprehensive and will lead to a complete answer.
    """
    
    planning_response = call_openrouter(
        prompt=planning_prompt,
        model=model,
        temperature=0.3,
        max_tokens=1000
    )
    
    if not planning_response.get("success", False):
        return f"Error in planning: {planning_response.get('error', 'Unknown error')}", {"error": planning_response.get('error')}
    
    plan = extract_text_response(planning_response)
    
    # Step 2: Parse the plan into executable steps
    # This is a simple parser that looks for Step N: patterns
    step_pattern = r'Step\s+(\d+):(.*?)(?:Step\s+\d+:|$)'  
    step_matches = re.finditer(step_pattern, plan, re.DOTALL)
    
    parsed_steps = []
    for match in step_matches:
        step_num = match.group(1)
        step_content = match.group(2).strip()
        
        # Extract tool and parameters
        tool_match = re.search(r'Tool:\s*([\w\s]+)', step_content, re.IGNORECASE)
        param_match = re.search(r'Parameters:\s*(.*?)(?:$|\n\n)', step_content, re.DOTALL | re.IGNORECASE)
        
        tool = tool_match.group(1).strip() if tool_match else None
        params = param_match.group(1).strip() if param_match else None
        
        # Skip steps with no tool (informational steps)
        if tool and tool.lower() != "none":
            parsed_steps.append({
                "step_num": int(step_num),
                "content": step_content,
                "tool": tool,
                "params": params
            })
    
    # Step 3: Execute each step in the plan
    execution_results = []
    tool_outputs = {}
    
    for step in parsed_steps:
        # Prepare execution prompt
        execution_prompt = f"""
        You are an AI assistant that executes specific steps in a plan.
        
        Question: {question}
        
        Current step to execute: {step['content']}
        
        You need to extract the exact tool name and parameters from this step. 
        Return ONLY the tool call in this format: TOOL: tool_name(parameter1="value", parameter2="value")
        
        Available tools:
        - search_wiki(query): Search for information in a Wikipedia-like database
        - calculator(expression): Evaluate a mathematical expression
        - lookup_data(dataset, query): Look up information in a dataset (available datasets: countries, planets, elements)
        """
        
        execution_response = call_openrouter(
            prompt=execution_prompt,
            model=model,
            temperature=0.3,
            max_tokens=300
        )
        
        if not execution_response.get("success", False):
            execution_results.append({
                "step": step,
                "error": execution_response.get('error', 'Unknown error'),
                "status": "failed"
            })
            continue
        
        tool_call = extract_text_response(execution_response)
        
        # Extract the tool and parameters
        tool_match = re.search(r'TOOL:\s*(\w+)\((.+?)\)', tool_call)
        if not tool_match:
            execution_results.append({
                "step": step,
                "error": "Could not parse tool call",
                "tool_call": tool_call,
                "status": "failed"
            })
            continue
        
        tool_name = tool_match.group(1).strip()
        args_str = tool_match.group(2).strip()
        
        # Parse the arguments
        args = {}
        for arg_match in re.finditer(r'(\w+)\s*=\s*"([^"]*)"|\s*(\w+)\s*=\s*\'([^\']*)\'', args_str):
            if arg_match.group(1):
                arg_name = arg_match.group(1)
                arg_value = arg_match.group(2)
            else:
                arg_name = arg_match.group(3)
                arg_value = arg_match.group(4)
            args[arg_name] = arg_value
        
        # Execute the tool
        if tool_name in tools:
            try:
                tool_result = tools[tool_name](**args)
                step_key = f"step_{step['step_num']}"
                tool_outputs[step_key] = tool_result
                
                execution_results.append({
                    "step": step,
                    "tool": tool_name,
                    "args": args,
                    "result": tool_result,
                    "status": "success"
                })
            except Exception as e:
                execution_results.append({
                    "step": step,
                    "tool": tool_name,
                    "args": args,
                    "error": str(e),
                    "status": "failed"
                })
        else:
            execution_results.append({
                "step": step,
                "tool": tool_name,
                "error": f"Tool '{tool_name}' not found",
                "status": "failed"
            })
    
    # Step 4: Synthesize the results into a final answer
    synthesis_prompt = f"""
    You are an AI assistant that synthesizes information to answer questions.
    
    Question: {question}
    
    Plan:
    {plan}
    
    Execution Results:
    {"..".join([f"Step {result['step']['step_num']}: {result['step']['content']}..Result: {result.get('result', result.get('error', 'No result'))}" for result in execution_results])}
    
    Based on these results, provide a comprehensive answer to the original question.
    """
    
    synthesis_response = call_openrouter(
        prompt=synthesis_prompt,
        model=model,
        temperature=0.5,
        max_tokens=800
    )
    
    if not synthesis_response.get("success", False):
        return f"Error in synthesis: {synthesis_response.get('error', 'Unknown error')}", {
            "plan": plan,
            "execution_results": execution_results,
            "error": synthesis_response.get('error')
        }
    
    final_answer = extract_text_response(synthesis_response)
    
    return final_answer, {
        "plan": plan,
        "execution_results": execution_results,
        "tool_outputs": tool_outputs
    }

# Test the planning-execution loop
planning_question = "Which planet has more moons, Jupiter or Saturn, and how many does each have?"
planning_answer, planning_details = planning_execution_loop(planning_question, tools_dict)

print(f"Question: {planning_question}\n")
print("Generated Plan:")
print(planning_details["plan"])
print("\nExecution Results:")
for i, result in enumerate(planning_details["execution_results"]):
    print(f"\nStep {result['step']['step_num']}:")
    print(f"Tool: {result.get('tool', 'Unknown')}")
    print(f"Arguments: {result.get('args', 'None')}")
    print(f"Status: {result['status']}")
    if 'result' in result:
        print(f"Result: {result['result']}")
    elif 'error' in result:
        print(f"Error: {result['error']}")

print("\nFinal Answer:")
print(planning_answer)

Question: Which planet has more moons, Jupiter or Saturn, and how many does each have?

Generated Plan:
Step 1: Gather information about the number of moons for Jupiter.
Tool: lookup_data
Parameters: planets, "Jupiter"

Step 2: Gather information about the number of moons for Saturn.
Tool: lookup_data
Parameters: planets, "Saturn"

Step 3: Compare the number of moons between Jupiter and Saturn.
Tool: None
Parameters: None
Purpose: To determine which planet has more moons based on the data retrieved in the previous steps.

Step 4: Compile the results into a clear answer format.
Tool: None
Parameters: None
Purpose: To present the findings in a concise manner, stating which planet has more moons and the specific counts for each.

Execution Results:

Step 1:
Tool: lookup_data
Arguments: {'dataset': 'planets', 'query': 'Jupiter'}
Status: success
Result: Jupiter:
- Position: 5
- Type: Gas Giant
- Moons: 79
- Rings: Yes

Step 3:
Tool: lookup_data
Arguments: {}
Status: failed
Error: lookup_dat

## 5. Reflection and Self-Correction Mechanisms

Let's implement a reflection mechanism that allows the agent to recognize and correct its own mistakes.

In [11]:
def reflective_agent(question: str, tools: Dict[str, Callable], model: str = "openai/gpt-4o-mini-2024-07-18") -> Tuple[str, Dict[str, Any]]:
    """An agent that can reflect on and correct its own reasoning.
    
    Args:
        question: The user's question
        tools: Dictionary mapping tool names to functions
        model: LLM model to use
        
    Returns:
        Tuple of (final answer, dict with reflection details)
    """
    # First, get an initial answer using the ReAct pattern
    print("Getting initial answer...")
    initial_answer, steps = raw_react(question, tools, max_iterations=3, model=model)
    
    # Prepare a prompt that shows the full reasoning process
    reasoning_process = ""
    for i, step in enumerate(steps):
        reasoning_process += f"\nStep {i+1}:\n{step['reasoning']}\n"
        if step["type"] == "action" and "observation" in step:
            reasoning_process += f"{step['observation']}\n"
    
    # Reflection prompt
    reflection_prompt = f"""
    You are an AI assistant that can reflect on and correct reasoning.
    
    Question: {question}
    
    Below is a reasoning process that led to an answer. Carefully analyze this reasoning for errors, 
    missing steps, or faulty logic. Then provide a reflection that identifies any issues and suggests improvements.
    
    Reasoning Process:
    {reasoning_process}
    
    Initial Answer: {initial_answer}
    
    Reflection Instructions:
    1. Identify any errors or flaws in the reasoning
    2. Note any missing information or steps that should have been taken
    3. Assess whether the final answer is correct and complete
    4. Suggest specific improvements to the reasoning process
    
    Format your reflection as follows:
    
    REFLECTION:
    [Your detailed reflection here]
    
    CORRECTIONS NEEDED: [Yes/No]
    
    IMPROVED ANSWER: [Provide a corrected answer only if corrections are needed]
    """
    
    print("Reflecting on initial answer...")
    reflection_response = call_openrouter(
        prompt=reflection_prompt,
        model=model,
        temperature=0.5,
        max_tokens=800
    )
    
    if not reflection_response.get("success", False):
        return f"Error in reflection: {reflection_response.get('error', 'Unknown error')}", {"error": reflection_response.get('error')}
    
    reflection = extract_text_response(reflection_response)
    
    # Extract whether corrections are needed
    corrections_match = re.search(r'CORRECTIONS NEEDED:\s*(Yes|No)', reflection, re.IGNORECASE)
    corrections_needed = False
    if corrections_match:
        corrections_needed = corrections_match.group(1).lower() == "yes"
    
    # Extract the improved answer if available
    improved_answer = initial_answer
    improved_match = re.search(r'IMPROVED ANSWER:\s*(.*?)(?:$)', reflection, re.DOTALL)
    if improved_match and corrections_needed:
        improved_answer = improved_match.group(1).strip()
    
    # If there are still issue, do a second pass with more specific instructions
    if corrections_needed:
        print("Corrections needed. Performing a second reasoning pass...")
        
        # Extract reflection content
        reflection_content = ""
        reflection_match = re.search(r'REFLECTION:\s*(.*?)(?:CORRECTIONS NEEDED:|$)', reflection, re.DOTALL)
        if reflection_match:
            reflection_content = reflection_match.group(1).strip()
        
        # Create a new prompt with the reflection as guidance
        improved_prompt = f"""
        You are an AI assistant answering a user's question with careful reasoning.
        
        Question: {question}
        
        A previous attempt to answer this question had some issues. Here's a reflection on those issues:
        
        {reflection_content}
        
        With this feedback in mind, approach the question again. Use the available tools if needed:
        - search_wiki(query): Search for information in a Wikipedia-like database
        - calculator(expression): Evaluate a mathematical expression
        - lookup_data(dataset, query): Look up information in a dataset (available datasets: countries, planets, elements)
        
        Show your complete reasoning process and provide a correct, comprehensive answer.
        """
        
        final_response = call_openrouter(
            prompt=improved_prompt,
            model=model,
            temperature=0.5,
            max_tokens=800
        )
        
        if final_response.get("success", False):
            final_answer = extract_text_response(final_response)
        else:
            final_answer = f"Error in final correction: {final_response.get('error', 'Unknown error')}"
    else:
        # No corrections needed, use the improved or initial answer
        final_answer = improved_answer
    
    return final_answer, {
        "initial_answer": initial_answer,
        "reflection": reflection,
        "corrections_needed": corrections_needed,
        "improved_answer": improved_answer,
        "steps": steps
    }

# Test the reflective agent with a question that might have reasoning flaws
reflection_question = "If Saturn has 82 moons and Jupiter has 79 moons, what percentage more moons does Saturn have compared to Jupiter?"
reflection_answer, reflection_details = reflective_agent(reflection_question, tools_dict)

print(f"\nQuestion: {reflection_question}\n")
print("Initial Answer:")
print(reflection_details["initial_answer"])
print("\nReflection:")
print(reflection_details["reflection"])
print("\nFinal Answer:")
print(reflection_answer)

Getting initial answer...
Reflecting on initial answer...
Corrections needed. Performing a second reasoning pass...

Question: If Saturn has 82 moons and Jupiter has 79 moons, what percentage more moons does Saturn have compared to Jupiter?

Initial Answer:
Saturn has approximately 3.8% more moons than Jupiter.

Reflection:
REFLECTION:
The reasoning process is mostly correct, but there is a significant error in the calculation of the percentage. The steps to find the difference in the number of moons and to express that difference as a percentage of Jupiter's moons are correctly outlined. However, the calculator action should yield a different result than what was stated as the final answer.

Let's break it down:

1. The difference in the number of moons is calculated correctly: 
   - 82 (Saturn) - 79 (Jupiter) = 3 moons.

2. To find the percentage more moons Saturn has compared to Jupiter, the formula used is correct:
   - Percentage more = (Difference / Jupiter's moons) * 100
   - Th

## 6. Combining Multiple Reasoning Frameworks

Now, let's combine multiple reasoning frameworks into a single, powerful agent. This agent will use:

1. Chain of Thought to break down problems
2. Planning to create a structured approach
3. ReAct to dynamically interact with tools
4. Reflection to catch and correct errors

In [12]:
def combined_reasoning_agent(question: str, tools: Dict[str, Callable], model: str = "openai/gpt-4o-mini-2024-07-18") -> Tuple[str, Dict[str, Any]]:
    """An agent that combines multiple reasoning frameworks.
    
    Args:
        question: The user's question
        tools: Dictionary mapping tool names to functions
        model: LLM model to use
        
    Returns:
        Tuple of (final answer, dict with process details)
    """
    # Step 1: Use Chain of Thought to break down the problem
    print("Step 1: Problem Analysis using Chain of Thought")
    cot_prompt = f"""
    You are an AI assistant that breaks down problems step by step.
    
    Question: {question}
    
    Before jumping into the solution, carefully analyze the problem:
    1. What is the core question being asked?
    2. What information do we need to answer it?
    3. What potential approaches could we take?
    4. Are there any potential complications or edge cases?
    
    Structure your analysis clearly.
    """
    
    cot_response = call_openrouter(
        prompt=cot_prompt,
        model=model,
        temperature=0.3,
        max_tokens=800
    )
    
    if not cot_response.get("success", False):
        return f"Error in problem analysis: {cot_response.get('error', 'Unknown error')}", {"error": cot_response.get('error')}
    
    problem_analysis = extract_text_response(cot_response)
    
    # Step 2: Create a plan based on the analysis
    print("Step 2: Creating a structured plan")
    planning_prompt = f"""
    You are an AI assistant that creates clear, structured plans.
    
    Question: {question}
    
    Problem Analysis:
    {problem_analysis}
    
    Based on this analysis, create a step-by-step plan to answer the question.
    For each step, specify:
    1. The purpose of the step
    2. Whether a tool is needed (and if so, which one)
    3. What we expect to learn from this step
    
    Available tools:
    - search_wiki(query): Search for information in a Wikipedia-like database
    - calculator(expression): Evaluate a mathematical expression
    - lookup_data(dataset, query): Look up information in a dataset (available datasets: countries, planets, elements)
    
    Format your plan as numbered steps.
    """
    
    planning_response = call_openrouter(
        prompt=planning_prompt,
        model=model,
        temperature=0.3,
        max_tokens=800
    )
    
    if not planning_response.get("success", False):
        return f"Error in planning: {planning_response.get('error', 'Unknown error')}", {
            "problem_analysis": problem_analysis,
            "error": planning_response.get('error')
        }
    
    plan = extract_text_response(planning_response)
    
    # Step 3: Execute the plan using ReAct
    print("Step 3: Executing the plan using ReAct")
    react_prompt = f"""
    You are an AI assistant that follows plans while being flexible enough to adapt when needed.
    
    Question: {question}
    
    Problem Analysis:
    {problem_analysis}
    
    Plan:
    {plan}
    
    Your task is to execute this plan step by step. For each step:
    1. Think about what needs to be done
    2. Use tools when needed in the format: ACTION: tool_name(parameter1="value", parameter2="value")
    3. Review the results and continue to the next step
    
    Available tools:
    - search_wiki(query): Search for information in a Wikipedia-like database
    - calculator(expression): Evaluate a mathematical expression
    - lookup_data(dataset, query): Look up information in a dataset (available datasets: countries, planets, elements)
    
    When you have the final answer, respond with: FINAL ANSWER: your complete answer here
    
    Begin executing the plan now.
    """
    
    # Use our raw_react function to execute this
    react_answer, react_steps = raw_react(react_prompt, tools, max_iterations=5, model=model)
    
    # Extract the final answer from the ReAct execution
    if "FINAL ANSWER:" in react_answer:
        execution_result = react_answer.split("FINAL ANSWER:")[1].strip()
    else:
        execution_result = react_answer
    
    # Step 4: Reflect on the result
    print("Step 4: Reflecting on the result")
    reflection_prompt = f"""
    You are an AI assistant that carefully reviews and improves answers.
    
    Question: {question}
    
    Problem Analysis:
    {problem_analysis}
    
    Plan:
    {plan}
    
    Execution Result:
    {execution_result}
    
    Please review this answer critically:
    1. Is the answer complete and directly responding to the question?
    2. Are there any errors in the reasoning or calculations?
    3. Is the answer presented clearly and concisely?
    4. Are there any important details or considerations missing?
    
    If improvements are needed, provide a revised answer that addresses any issues.
    
    Format your response as:
    
    REFLECTION:
    [Your reflection here]
    
    REVISED ANSWER:
    [The improved answer, or "The original answer is correct and complete." if no changes are needed]
    """
    
    reflection_response = call_openrouter(
        prompt=reflection_prompt,
        model=model,
        temperature=0.5,
        max_tokens=800
    )
    
    if not reflection_response.get("success", False):
        return execution_result, {
            "problem_analysis": problem_analysis,
            "plan": plan,
            "execution_result": execution_result,
            "react_steps": react_steps,
            "reflection_error": reflection_response.get('error')
        }
    
    reflection = extract_text_response(reflection_response)
    
    # Extract the revised answer
    revised_match = re.search(r'REVISED ANSWER:\s*(.*?)(?:$)', reflection, re.DOTALL)
    if revised_match:
        revised_answer = revised_match.group(1).strip()
        if "original answer is correct" in revised_answer.lower():
            final_answer = execution_result
        else:
            final_answer = revised_answer
    else:
        final_answer = execution_result
    
    return final_answer, {
        "problem_analysis": problem_analysis,
        "plan": plan,
        "execution_result": execution_result,
        "reflection": reflection,
        "react_steps": react_steps
    }

# Test the combined reasoning agent
combined_question = "If Earth has 1 moon, Mars has 2 moons, Jupiter has 79 moons, and Saturn has 82 moons, what is the average number of moons per planet for these 4 planets?"

combined_answer, combined_details = combined_reasoning_agent(combined_question, tools_dict)

print(f"\nQuestion: {combined_question}\n")
print("Problem Analysis:")
print(combined_details["problem_analysis"][:300] + "..." if len(combined_details["problem_analysis"]) > 300 else combined_details["problem_analysis"])
print("\nPlan:")
print(combined_details["plan"][:300] + "..." if len(combined_details["plan"]) > 300 else combined_details["plan"])
print("\nExecution Result:")
print(combined_details["execution_result"])
print("\nReflection:")
print(combined_details["reflection"][:300] + "..." if len(combined_details["reflection"]) > 300 else combined_details["reflection"])
print("\nFinal Answer:")
print(combined_answer)

Step 1: Problem Analysis using Chain of Thought
Step 2: Creating a structured plan
Step 3: Executing the plan using ReAct
Step 4: Reflecting on the result

Question: If Earth has 1 moon, Mars has 2 moons, Jupiter has 79 moons, and Saturn has 82 moons, what is the average number of moons per planet for these 4 planets?

Problem Analysis:
### Analysis of the Problem

1. **Core Question**: 
   The core question is asking for the average number of moons per planet for Earth, Mars, Jupiter, and Saturn. 

2. **Information Needed**: 
   To calculate the average number of moons per planet, we need:
   - The total number of moons for each o...

Plan:
### Step-by-Step Plan to Calculate the Average Number of Moons per Planet

1. **Step 1: Gather Moon Data for Each Planet**
   - **Purpose**: To collect the number of moons for Earth, Mars, Jupiter, and Saturn.
   - **Tool Needed**: `lookup_data(planets, query)` (to retrieve moon counts).
   - **Expe...

Execution Result:
The average number of moons

## 7. Summary and Best Practices

In this notebook, we've implemented several advanced reasoning frameworks from scratch without relying on specialized libraries:

1. **Chain of Thought (CoT)**: Encouraging step-by-step reasoning through prompt engineering
2. **ReAct Pattern**: Interleaving reasoning and action steps for dynamic problem-solving
3. **Planning-Execution Loop**: Creating a plan first, then executing each step methodically
4. **Reflection and Self-Correction**: Adding a review phase to identify and fix reasoning errors
5. **Combined Reasoning**: Integrating multiple frameworks for comprehensive problem-solving

### Best Practices for Implementing Reasoning Frameworks

1. **Clear Instructions**: Provide explicit formatting requirements in your prompts
2. **Structured Output**: Use consistent patterns (e.g., "ACTION:", "FINAL ANSWER:") for easier parsing
3. **Robust Parsing**: Use flexible regex patterns to handle variations in model outputs
4. **Error Handling**: Add fallbacks when parsing or tool execution fails
5. **Iterative Development**: Start simple and add complexity gradually
6. **Context Management**: Be mindful of prompt length to avoid exceeding context windows
7. **Temperature Settings**: Use lower temperature (0.0-0.3) for reasoning tasks, higher (0.5-0.7) for creative synthesis
8. **Model Selection**: More capable models generally perform better at complex reasoning

### When to Use Each Framework

- **Chain of Thought**: Use for math problems, logic puzzles, and other tasks requiring step-by-step reasoning
- **ReAct**: Best for exploratory tasks where the path isn't clear and interactive tools are needed
- **Planning-Execution**: Ideal for complex, multi-step tasks with a predictable workflow
- **Reflection**: Add when accuracy is critical and the cost of errors is high
- **Combined Approaches**: Use for the most challenging problems where a single approach may not be sufficient

By implementing these patterns from scratch, you gain a deeper understanding of how they work and can customize them for your specific use cases, without being constrained by library limitations.